In [ ]:
import pandas as pd
import json
import numpy as np

# Path dei file (aggiorna con i percorsi reali)
pairwise_file = 'result.csv'
groundtruth_file = 'data/ground_truth.csv'
output_json_file = 'filtered_ground_truth_3.json'

# Caricamento dei file
pairwise = pd.read_csv(pairwise_file)
groundtruth = pd.read_csv(groundtruth_file)

# Funzione di supporto per normalizzare i nomi di società (rimuove spazi e usa lowercase)
def normalize_name(name):
    return name.strip().lower() if isinstance(name, str) else str(name).lower()

# Funzione per ottenere una tupla ordinata, così che l'ordine non conti
def normalize_pair(row, col1, col2):
    left = normalize_name(row[col1])
    right = normalize_name(row[col2])
    return tuple(sorted([left, right]))

# Creiamo la colonna 'normalized_pair' in entrambi i dataset
pairwise['normalized_pair'] = pairwise.apply(
    lambda x: normalize_pair(x, 'company_name_left', 'company_name_right'), axis=1
)
groundtruth['normalized_pair'] = groundtruth.apply(
    lambda x: normalize_pair(x, 'company_a', 'company_b'), axis=1
)

# Convertiamo in insiemi per confronto
pairwise_set = set(pairwise['normalized_pair'])
groundtruth_set = set(groundtruth['normalized_pair'])

# Intersezione tra le coppie del dataset pairwise e quelle della ground truth (tutte, indipendentemente dalla label)
intersection = pairwise_set.intersection(groundtruth_set)

# Filtriamo la ground truth per ottenere solo le coppie con label 1 (cioè i match veri)
groundtruth_true_set = set(groundtruth[groundtruth['label'] == 1]['normalized_pair'])

# Calcoliamo il numero di coppie dell'intersezione che hanno label 1
tp = len(intersection.intersection(groundtruth_true_set))

# Calcolo delle metriche:
precision = tp / len(intersection) if len(intersection) else 0
recall = tp / len(groundtruth_true_set) if len(groundtruth_true_set) else 0
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) else 0

# Output dei risultati
print(f"Numero di coppie in pairwise: {len(pairwise_set)}")
print(f"Numero di coppie in ground truth: {len(groundtruth_set)}")
print(f'Numero di coppie in ground truth con label a 1: {len(groundtruth_true_set)}')
print(f"Numero di coppie in intersezione: {len(intersection)}")
print(f"Numero di coppie nell'intersezione con label 1: {tp}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1_score:.4f}")

# Creazione del JSON filtrato
filtered_gt = {"ground_truth": []}

# Iteriamo sulla ground truth e selezioniamo solo le coppie nell'intersezione
for _, row in groundtruth.iterrows():
    normalized = normalize_pair(row, 'company_a', 'company_b')
    if normalized in intersection:
        gt_entry = {
            "pairs": [
                {
                    "company-name": row["company_a"],
                    "industry": row["industry_a"].split("; ") if isinstance(row["industry_a"], str) else None,
                    "country": None if pd.isna(row["country_a"]) else row["country_a"],
                    "source": None if pd.isna(row["source_a"]) else row["source_a"]
                },
                {
                    "company-name": row["company_b"],
                    "industry": row["industry_b"].split("; ") if isinstance(row["industry_b"], str) else None,
                    "country": None if pd.isna(row["country_b"]) else row["country_b"],
                    "source": None if pd.isna(row["source_b"]) else row["source_b"]
                }
            ],
            "match": True if row["label"] == 1 else False
        }
        filtered_gt["ground_truth"].append(gt_entry)

# Salvataggio su file JSON
with open(output_json_file, "w", encoding="utf-8") as json_file:
    json.dump(filtered_gt, json_file, indent=4, ensure_ascii=False)

print(f"✅ File JSON '{output_json_file}' generato con successo!")
